In [38]:
import json
import os
import datetime
import pandas as pd
from datetime import datetime
from dateutil import tz

In [27]:
json_dir = r"C:\Users\Bschuster\Documents\hivergent_analytics\ETL\dev\api_extracts\btc\json"
satoshi_to_BTC = 100000000

In [28]:
os.listdir(json_dir)

['btc_501798_501802.json',
 'btc_501803_501807.json',
 'btc_501808_501812.json',
 'btc_501813_501817.json',
 'btc_501818_501822.json',
 'btc_501823_501827.json',
 'btc_501828_501832.json',
 'btc_501833_501837.json',
 'btc_501838_501842.json',
 'btc_501843_501847.json',
 'btc_501848_501852.json',
 'btc_501853_501857.json',
 'btc_501858_501862.json',
 'btc_501863_501867.json',
 'btc_501868_501872.json',
 'btc_501873_501877.json',
 'btc_501878_501882.json',
 'btc_501883_501887.json',
 'btc_501888_501892.json',
 'btc_501893_501897.json',
 'btc_501898_501902.json',
 'btc_501903_501907.json',
 'btc_501908_501912.json',
 'btc_501913_501917.json',
 'btc_501918_501922.json',
 'btc_501923_501927.json',
 'btc_501928_501932.json',
 'btc_501933_501937.json',
 'btc_501938_501942.json',
 'btc_501943_501947.json',
 'btc_501948_501952.json',
 'btc_501953_501957.json',
 'btc_501958_501960.json']

In [29]:
test_file = json_dir + r'/' + os.listdir(json_dir)[0]
with open (test_file, 'r') as f:
    data = f.read()

In [30]:
test_file_name = os.listdir(json_dir)[0]
os.path.splitext(test_file_name)[0]

'btc_501798_501802'

In [31]:
json_data = json.loads(data)
test_block = json_data['501798']

In [32]:
len(json_data)

5

In [6]:
test_block['data']['list'][0]['is_coinbase'] == True

True

In [11]:
def process_block_transactions(block_tx):
    '''
    Takes the raw transaction feed from btc.com tx api and 
    processes the txs into the data format.
    '''
    processed_transactions = []
    raw_transactions = block_tx['data']['list']
    #
    
    for i in raw_transactions:
        next_tx = {}
        
        #We are creating one 'transaction' per sender and ignoring
        #the receiver. We will build the attributes that will be
        #true for all transactions, then have a final loop to 
        #go through the sender specfiic details.
        
        #transaction hash
        next_tx['transaction_hash'] = i['hash']
        
        #unixtimestamp
        timestamp = i['block_time']
        next_tx['unixtimestamp'] = timestamp
        
        #datetime
        next_tx['datetime'] = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
        
        #blockchain network name
        next_tx['blockchain_network_name'] = 'bitcoin'
        
        #sent currency name
        next_tx['sent_currency_name'] = 'btc'
        
        #status
        #if it's included in the BTC blockchain, it succeeded
        next_tx['status'] = 1
        
        #If Coinbase, sender is coinbase and will be sent as such
        if i['is_coinbase'] == True:
            #id
            next_tx['id'] = i['hash']
            
            #Sender Address
            next_tx['sender_address'] = 'coinbase'
            
            #Transaction Type
            next_tx['transaction_type_name'] = 'payment'
            
            #Transaction Subtype
            next_tx['transaction_subtype_name'] = 'coinbase'
            
            #fee
            next_tx['fee'] = 0 
            
            #Sent Currency Amount
            next_tx['sent_currency_amount'] = i['outputs_value']/float(satoshi_to_BTC)
            
        else:
            
            for n, send_tx in enumerate(i['inputs']):
                #id
                next_tx['id'] = str(i['hash']) + '_' + str(n)
                
                #Sender Address
                next_tx['sender_address'] = send_tx['prev_addresses'][0]
                
                #Transaction Type
                next_tx['transaction_type_name'] = 'payment'
                
                #Transaction Subtype
                #Will likely need to be re-worked to get more complexity
                next_tx['transaction_subtype_name'] = 'simple_payment'
                
                #virtual fee calc
                #Virtual fee calculated because each send does not have a few, but the whole tx
                #We divide the fee by it's proportional amount of BTC sent
                #otherwise it's possible to get a negative amount sent :( 
                total_value_of_inputs = i['inputs_value']
                percent_of_total_tx_sent = send_tx['prev_value']/float(total_value_of_inputs)
                virtual_fee = i['fee'] * percent_of_total_tx_sent
                next_tx['fee'] = virtual_fee/float(satoshi_to_BTC)
                
                #Sent Currency Amount   
                #With a virtual fee, we need a virtual sent amount
                #Real sent minus virtual fee = virtual sent
                virtual_sent = send_tx['prev_value'] - virtual_fee
                next_tx['sent_currency_amount'] = virtual_sent/float(satoshi_to_BTC)
    
        processed_transactions.append(next_tx)
    
    return processed_transactions

In [17]:
block_transactions = []
for i in range(len(json_data)):
    next_block_key = json_data.keys()[i]
    next_block = json_data[next_block_key]
    next_block_tx = process_block_transactions(next_block)
    block_transactions += next_block_tx

In [35]:
test_trans = test_block['data']['list'][2]

In [36]:
test_trans

{u'block_height': 501798,
 u'block_time': 1514678885,
 u'confirmations': 4308,
 u'created_at': 1514678615,
 u'fee': 315720,
 u'hash': u'ca1ee6f2d8590c4f1a5175a09e79295056aaf76684513a08619c43c02203401e',
 u'inputs': [{u'prev_addresses': [u'1B3kww63wgHvYBtv3ja4gZFRGkZwNyAT4X'],
   u'prev_position': 0,
   u'prev_tx_hash': u'e20821ea2c03d4c0b83a2896911a52d8f158302f053c75082f7f51d682af30fd',
   u'prev_type': u'P2PKH',
   u'prev_value': 72188980,
   u'sequence': 4294967295L}],
 u'inputs_count': 1,
 u'inputs_value': 72188980,
 u'is_coinbase': False,
 u'is_double_spend': False,
 u'is_sw_tx': False,
 u'lock_time': 0,
 u'outputs': [{u'addresses': [u'1N9tfCZY1Cw6oHyYXr6ArUR2Hetp4DVLPL'],
   u'spent_by_tx': u'cd9e6e4bb8eb9e93cdce910133da024282b7ca236096d3618e25caa8f6835f2e',
   u'spent_by_tx_position': 51,
   u'type': u'P2PKH',
   u'value': 15996900},
  {u'addresses': [u'1DXYx9DwqCuP2YydTQZWvJNsxdCPekXiEf'],
   u'spent_by_tx': u'a1f7693eea51c59d4b6791f747c61a13a1ec9dcc45f9ca3f9347b78bd92eb758',
  

In [39]:
time_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')

In [47]:
utc = datetime.utcfromtimestamp(test_trans['block_time'])
utc = utc.replace(tzinfo=from_zone)

In [48]:
utc.strftime("%Y-%m-%d %H:%M:%S")

'2017-12-31 00:08:05'